### Contents

[Imports](#Imports)

[bottom](#bottom)

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pprint import pprint
import sys 
import json 
import itertools 
from tqdm import tqdm
from datetime import datetime, date, timedelta
from load_dotenv import load_dotenv
load_dotenv() # OPENAI_API_KEY

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.display import display, clear_output

In [3]:
matplotlib.rcParams['figure.figsize'] = (7, 7)
sns.set_style('whitegrid')

---

In [4]:
import openai
from openai import OpenAI

In [5]:
# os.environ['OPENAI_API_KEY'] = 'xxxx'

### Q1. Prepare Documents
Import documents.json, read the file and prepare the dataset:

```python
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

len(documents)
```

How many records we have in the pre-processed "documents"?

- 1000
- 1051
- 901
- 948

In [5]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
docs_raw = docs_response.json()

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

len(documents)

948

### Q2. Create Embeddings using Pretrained Models

Import sentence transformer library. Please review the Sentence Transformer pretrained documentation here: [https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

```python
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"

from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-MiniLM-L12-v2")
```


In [6]:
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-MiniLM-L12-v2")

In [28]:
questions_embeddings = model.encode([info['question'] for info in documents])
print(f"questions_embeddings (type: {type(questions_embeddings)}) {questions_embeddings.shape[0]:,} x {questions_embeddings.shape[1]:,}")

for i, doc in tqdm(enumerate(documents)):
    doc['question_vector'] = questions_embeddings[i].tolist()

questions_embeddings (type: <class 'numpy.ndarray'>) 948 x 384


948it [00:00, 124524.76it/s]


In [23]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': '5cca4a99c596', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'qGWRjdWnRGKNbgCTgHPxAQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [24]:
### compared to index setting from 1st homework:
# es_client.indices.create(
#     index='course-questions',
#     body={
#         "settings": {
#             "number_of_shards": 1,
#             "number_of_replicas": 0
#         },
#         "mappings": {
#             "properties": {
#                 "text": {"type": "text"},
#                 "section": {"type": "text"},
#                 "question": {"type": "text"},
#                 "course": {"type": "keyword"} 
#             }
#         }
#     }
# )
index_name = "course-questions"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(
    index=index_name, 
    body={
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "keyword"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} ,
                "question_vector":{"type":"dense_vector","dims": 384,"index":True,"similarity": "cosine"
            },
            }
        }
    }
)
es_client

<Elasticsearch(['http://localhost:9200'])>

In [32]:
# adding documents to index
for doc in tqdm(documents):
    try:
        es_client.index(
            index=index_name, 
            document=doc,
        )
    except Exception as e:
        print(e)

100%|██████████| 948/948 [00:02<00:00, 340.74it/s]


In [38]:
search_term = "how to enrol to course?"
vector_search_term = model.encode(search_term)

query = {
    "field": "question_vector",
    "query_vector": vector_search_term,
    "k": 3,
    "num_candidates": 10_000, 
}
res = es_client.search(
    index="course-questions", 
    knn=query,
    source=["text","section","question","course"]
)
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'gyN2hJABz2SCwBl9ujll',
  '_score': 0.85204804,
  '_source': {'question': 'Course - What are the prerequisites for this course?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites'}},
 {'_index': 'course-questions',
  '_id': 'pCN2hJABz2SCwBl9ujnz',
  '_score': 0.8381672,
  '_source': {'question': 'How can we contribute to the course?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.'}},
 {'_index': 'course-questions',
  '_id': 'RiN2hJABz2SCwBl9vzvZ',
  '_score': 0.8284229,
  '_source': {'question': 'I just joined. What should I do next? How can I access course materials?',
   'course': 'machine-learning-zoomcamp',
   'section': 'Ge

In [35]:
search_term = "What is docker and how to use docker-compose?"
vector_search_term = model.encode(search_term)

query = {
    "field": "question_vector",
    "query_vector": vector_search_term,
    "k": 3,
    "num_candidates": 10_000, 
}
res = es_client.search(
    index="course-questions", 
    knn=query,
    source=["text","section","question","course"]
)
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': '2yN2hJABz2SCwBl9vjq4',
  '_score': 0.82894623,
  '_source': {'question': 'Spark docker-compose setup',
   'course': 'data-engineering-zoomcamp',
   'section': 'Module 5: pyspark',
   'text': 'To run spark in docker setup\n1. Build bitnami spark docker\na. clone bitnami repo using command\ngit clone https://github.com/bitnami/containers.git\n(tested on commit 9cef8b892d29c04f8a271a644341c8222790c992)\nb. edit file `bitnami/spark/3.3/debian-11/Dockerfile` and update java and spark version as following\n"python-3.10.10-2-linux-${OS_ARCH}-debian-11" \\\n"java-17.0.5-8-3-linux-${OS_ARCH}-debian-11" \\\nreference: https://github.com/bitnami/containers/issues/13409\nc. build docker image by navigating to above directory and running docker build command\nnavigate cd bitnami/spark/3.3/debian-11/\nbuild command docker build -t spark:3.3-java-17 .\n2. run docker compose\nusing following file\n```yaml docker-compose.yml\nversion: \'2\'\nservices:\nspark:\n

In [52]:
knn_query = {
    "field": "question_vector",
    "query_vector":  vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {
                "section": "General course-related questions"
            },
        },
    knn=knn_query,
    size=5
)

response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'gyN2hJABz2SCwBl9ujll',
  '_score': 3.4953294,
  '_source': {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
   'section': 'General course-related questions',
   'question': 'Course - What are the prerequisites for this course?',
   'course': 'data-engineering-zoomcamp',
   'question_vector': [0.043121155351400375,
    0.05837034434080124,
    0.005023022647947073,
    0.028570635244250298,
    -0.022065389901399612,
    0.004731929861009121,
    -0.02545989118516445,
    -0.0008887685253284872,
    -0.16603532433509827,
    -0.004694784991443157,
    -0.05639379471540451,
    -0.10678312927484512,
    -0.06021315976977348,
    0.03971245139837265,
    -0.063615582883358,
    0.03923672065138817,
    -0.004405468236654997,
    -0.008392675779759884,
    0.14483694732189178,
    -0.07729381322860718,
    0.01507660560309887,
    0.020862001925706863,
    0.01940772868692875,
    -0.031069358810782433,
    0.0079319523647

### Q1. Getting the embeddings model
First, we will get the embeddings model `multi-qa-distilbert-cos-v1` from the Sentence Transformer library

```python
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embedding for this user question:

user_question = "I just discovered the course. Can I still join it?"
What's the first value of the resulting vector?

- -0.24
- -0.04
- 0.07
- 0.27

In [5]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

/Users/antonandreytsev/llm_zoomcamp_venv3.9.7/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
user_question = "I just discovered the course. Can I still join it?"
embedding_model.encode(user_question)[0]

0.07822261

Prepare the documents
Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:
```python
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
```

We will use only a subset of the questions - the questions for "machine-learning-zoomcamp". After filtering, you should have only 375 documents



In [8]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
print(len(documents))

948


### Q2. Creating the embeddings
Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix X:

- Create a list embeddings
- Iterate over each document
- `qa_text = f'{question} {text}'`
- compute the embedding for qa_text, append to embeddings
- At the end, let `X = np.array(embeddings)` (`import numpy as np`)
- What's the shape of X? (`X.shape`). Include the parantheses.

In [15]:
embeds_array = embedding_model.encode([
    f"{doc_meta['question']} {doc_meta['text']}"
    for doc_meta in documents
])
print(embeds_array.shape)

(948, 768)


### Q3. Search

We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). This means that in order to compute the coside similarity, it's sufficient to multiply the matrix X by the vector v:

`scores = X.dot(v)`
What's the highest score in the results?

- 65.0
- 6.5
- 0.65
- 0.065

In [35]:
from scipy.spatial.distance import cdist
(
    1 - cdist(
        XA=embedding_model.encode(user_question).reshape(1, -1),
        XB=embeds_array,
        metric='cosine'
    ).ravel()
).max()

0.6506573641439212

In [37]:
(embeds_array @ embedding_model.encode(user_question)).max()

0.65065736

### Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

```python
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)
```

If you don't understand how the search function work:

Ask ChatGTP or any other LLM of your choice to explain the code
Check our pre-course workshop about implementing a search engine here
(Note: you can replace argsort with argpartition to make it a lot faster)



In [38]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [41]:
import typing as t 
import numpy.typing as npt 

DocumentType = t.Dict[str, str]

class VectorSearchEngine():
    def __init__(self, documents: t.List[DocumentType], embeddings: npt.NDArray[float]):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query: np.ndarray, num_results=10) -> t.List[DocumentType]:
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

print("user question:")
print(user_question, end='\n'*2)
query_vector = embedding_model.encode(user_question)
search_engine = VectorSearchEngine(documents=documents, embeddings=embeds_array)
search_engine.search(query_vector, num_results=5)

user question:
I just discovered the course. Can I still join it?



[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'id': '7842b56a'},
 {'text': 'Star the r

### Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will use the hitrate metric for evaluation.

First, load the ground truth dataset:

```python
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
```

Now use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5.

What did you get?

- 0.93
- 0.73
- 0.53
- 0.33

---
Hit Rate (HR) or Recall at k:

Measures the proportion of queries for which at least one relevant document is retrieved in the top k results.
Formula: HR@k = (Number of queries with at least one relevant document in top k) / |Q|


In [42]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [58]:
# computing hit-rate
denominator = len(ground_truth)
numerator = 0
num_results = 5
for doc_meta in tqdm(ground_truth):
    question, relevant_doc = doc_meta['question'], doc_meta['document']
    question_vec = embedding_model.encode(question)
    results = search_engine.search(question_vec, num_results)
    retrieved_documents = [doc_meta_['id'] for doc_meta_ in results]
    if relevant_doc in retrieved_documents:
        numerator += 1
print(numerator / denominator)

100%|██████████| 1830/1830 [00:43<00:00, 42.45it/s]

0.9218579234972678


### Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

- Create the index with the same settings as in the module (but change the dimensions)
- Index the embeddings (note: you've already computed them)

After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?



In [ ]:
embeds_array = embedding_model.encode([
    f"{doc_meta['question']} {doc_meta['text']}"
    for doc_meta in documents
])
print(embeds_array.shape)

for i, doc_meta in tqdm(enumerate(documents)):
    doc_meta['question_and_text_vector'] = embeds_array[i].tolist()

In [69]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': '5cca4a99c596', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'qGWRjdWnRGKNbgCTgHPxAQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [102]:
index_name = "course-questions"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(
    index=index_name, 
    body={
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {"type": "text"},
                "section": {"type": "keyword"},
                "question": {"type": "text"},
                "course": {"type": "keyword"} ,
                #"id": {'type': "text"},
                "question_and_text_vector":{
                    "type":"dense_vector",
                    "dims": embeds_array.shape[1],
                    "index":True,
                    "similarity": "cosine"
                },
            }
        }
    }
)
es_client

<Elasticsearch(['http://localhost:9200'])>

In [103]:
# adding documents to index
for doc in tqdm(documents):
    try:
        es_client.index(
            index=index_name, 
            document=doc,
        )
    except Exception as e:
        print(e)

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [00:02<00:00, 364.13it/s]


In [104]:
search_term = "I just discovered the course. Can I still join it?"
vector_search_term = embedding_model.encode(search_term)

query = {
    "field": "question_and_text_vector",
    "query_vector": vector_search_term,
    "k": 3,
    "num_candidates": 10_000, 
}
res = es_client.search(
    index="course-questions", 
    knn=query,
    source=["text","section","question","course",'id']
)
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'XyNesZABz2SCwBl9wkZJ',
  '_score': 0.8253288,
  '_source': {'question': 'The course has already started. Can I still join it?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
   'id': 'ee58a693'}},
 {'_index': 'course-questions',
  '_id': 'oCNesZABz2SCwBl9vUR_',
  '_score': 0.7873188,
  '_source': {'question': 'Course - Can I still join the course after the start date?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': "Yes, even if you don't register, you're still e

### Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between the query and ALL the vectors in our database. Usually this is not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster.

Let's evaluate how worse the results are when we switch from exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

- 0.93
- 0.73
- 0.53
- 0.33

In [108]:
from elasticsearch import Elasticsearch

def search_in_elastic(
    search_query: str,
    embedder: t.Callable[[str], npt.NDArray],
    elastic_client: Elasticsearch,
    query_params: t.Dict[t.Any, t.Any],
) -> t.List[DocumentType]:
    vector_search_term = embedder(search_query)
    res = elastic_client.search(
        index="course-questions", 
        knn={**query_params, "query_vector": vector_search_term},
        source=["text","section","question","course","id"]
    )
    return res["hits"]["hits"]

In [110]:
# computing hit-rate for search with elastic
denominator = len(ground_truth)
numerator = 0
num_results = 5
for doc_meta in tqdm(ground_truth):
    question, relevant_doc = doc_meta['question'], doc_meta['document']
    results = search_in_elastic( 
        search_query=question,
        embedder=lambda query: embedding_model.encode(query),
        elastic_client=es_client,
        query_params={
            "field": "question_and_text_vector",
            "k": num_results,
            "num_candidates": 10_000, 
        },
    )
    retrieved_documents = [doc_meta_['_source']['id'] for doc_meta_ in results]
    if relevant_doc in retrieved_documents:
        numerator += 1
print(numerator / denominator)

  0%|          | 0/1830 [00:00<?, ?it/s]

100%|██████████| 1830/1830 [00:36<00:00, 50.67it/s]

0.9218579234972678


### bottom